In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.experimental.numpy as tnp
import glob

print("Using TensorFlow version %s" % tf.__version__)

Using TensorFlow version 2.6.0


In [2]:
tnp.experimental_enable_numpy_behavior()

In [3]:
tf.test.is_built_with_cuda()

True

In [4]:
tf.config.list_physical_devices('GPU')

2021-11-25 20:40:04.818475: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-25 20:40:04.822254: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-25 20:40:04.822399: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
from tensorflow.keras.layers import LSTM,Concatenate, Dense, Flatten ,Activation ,Input , BatchNormalization,Dropout , Bidirectional
from tensorflow.keras.models import Sequential, Model

In [6]:
building1_filenames = set(glob.glob("datasets/building1/*/*.feather"))
building2_filenames = set(glob.glob("datasets/building2/*/*.feather"))
building3_filenames = set(glob.glob("datasets/building3/*/*.feather"))
training_filenames = set(glob.glob("datasets/building1/known/*.feather"))
#test1_filenames = building1_filenames - training_filenames

In [7]:
import random
seed=10
random.seed(seed)
np.random.seed(seed)

In [8]:
# generate training samples

dfs = []
for file in training_filenames:
    df = pd.read_feather(file)
    dfs.append(df)


training_data = pd.concat(dfs, ignore_index=True)

#training_data.head()

orientation_acc = training_data[["iphoneAccX", "iphoneAccY", "iphoneAccZ"]]
orientation_gyro = training_data[[ "iphoneGyroX", "iphoneGyroY", "iphoneGyroZ"]]
orientation_mag = training_data[[ "iphoneMagX", "iphoneMagY", "iphoneMagZ"]] 

#orientation_data.head()

In [9]:
orientation_target = training_data[["orientX", "orientY", "orientZ", "orientW"]] 
orientation_target.head()

,orientX,orientY,orientZ,orientW
0,-0.004727,0.001564,-0.000030,0.999988
1,-0.004664,0.001573,-0.000002,0.999988
2,-0.004671,0.001567,0.000022,0.999988
3,-0.004677,0.001563,-0.000050,0.999988
4,-0.004675,0.001515,-0.000039,0.999988


In [10]:
acc_np = orientation_acc.to_numpy()
gyro_np = orientation_gyro.to_numpy()
mag_np = orientation_mag.to_numpy()

In [11]:
orient_np = orientation_target.to_numpy()
len(orient_np)

849119

In [12]:
def generate_training_samples(acc_, gyro_, mag_, orient_, batch_size = 64):
    while True:
        xa_batch = np.zeros((batch_size,100,3))
        xg_batch = np.zeros((batch_size,100,3))
        xm_batch = np.zeros((batch_size,100,3))
        y_theta_batch = np.zeros((batch_size,4))
        y_sigma_batch = np.finfo(np.float32).eps* np.ones((batch_size,6)) ## To remove
        y_batch = np.concatenate((y_theta_batch, y_sigma_batch), axis=1)
        #print(y_batch.shape)
        
        j = 0
        for i in range(len(orient_)):
            xa_batch[j,:,:] = acc_[i, :]
            xg_batch[j,:,:] = gyro_[i,:]
            xm_batch[j,:,:] = mag_[i,:]
            y_theta_batch[j,:4] = orient_[i,:]
            j += 1
            if j >= batch_size:
                j = 0              
                yield([xa_batch,xg_batch, xm_batch],[y_batch])
            #---------------------------------------------------------------------------------------

In [13]:
checkpoints_path = os.path.join(os.getcwd(),'datasets/CheckPoints_Model_OrientNet')

if not os.path.exists(checkpoints_path):
  os.mkdir(checkpoints_path)
check_point_template_path = os.path.join(checkpoints_path,'ckpt_epoch_{epoch:03d}_loss_{loss:.4f}_.hdf5') # vloss_{val_loss:.4f}
check_point_callback = tf.keras.callbacks.ModelCheckpoint(check_point_template_path)

import re #regular expresion
def get_all_checkpoints(checkpoints_path,checkpoint_main_name = 'ckpt'):
  all_checkpoints = [j for j in os.listdir(checkpoints_path) if j.startswith(checkpoint_main_name)]
  return all_checkpoints

def check_if_available_checkpoints(checkpoints_path,checkpoint_main_name = 'ckpt'):
  all_checkpoints = get_all_checkpoints(checkpoints_path,checkpoint_main_name)
  if(len(all_checkpoints) > 0):#checkpoints avilable
    all_checkpoints.sort(reverse=True)    
    latest_check_point = all_checkpoints[0]
    initial_epoch = int(re.search('epoch_(.*?)_', latest_check_point).group(1))    
  else:
    latest_check_point = None
    initial_epoch = 0
    
  return initial_epoch , latest_check_point


# Check if there are any check points initially
initial_epoch , latest_check_point = check_if_available_checkpoints(checkpoints_path)

In [14]:
orient_learning_rate = 0.0005 # convergence  within  20  epochs \ref{IDOL}

In [15]:

def quaterinion_inv(q):
    return (1/(tnp.sum(q**2))) * tnp.array([-1*q[0], -1*q[1], -1*q[2], q[3]])

def log_bar(q, batch_size = 64):
    #print(tf.shape(q[3]))
    log_batch = tnp.zeros((batch_size,3))
    log_list = tf.unstack(log_batch)

    for i in range(batch_size):
        if q[3][i] == 0:
            log_list[i] =tnp.array([0.0, 0.0, 0.0], dtype=tnp.float32)
            #print(tf.shape(log_batch[i]))
        else:
            v = tf.slice(q,
                   begin=[i,0],
                   size=[3,1])
            log_list[i] = tnp.arctan2(tnp.sum(tnp.square(v)), q[3][i])/tnp.sum(tnp.square(v)) * v
    return tf.stack(log_list)


def quaternion_multiply(quaternion1, quaternion0):
    #w0, x0, y0, z0 = quaternion0 # ERROR with TF
    #w1, x1, y1, z1 = quaternion1
    w0 = quaternion0[3]
    x0 = quaternion0[0]
    y0 = quaternion0[1]
    z0 = quaternion0[2]
    w1 = quaternion1[3]
    x1 = quaternion1[0]
    y1 = quaternion1[1]
    z1 = quaternion1[2]
    return tnp.array([-x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0,
                     x1 * w0 + y1 * z0 - z1 * y0 + w1 * x0,
                     -x1 * z0 + y1 * w0 + z1 * x0 + w1 * y0,
                     x1 * y0 - y1 * x0 + z1 * w0 + w1 * z0])

def boximinus(q1, q2):
    return 2 * log_bar(quaternion_multiply(quaterinion_inv(q2), q1))

In [16]:
# Loss Function
class OreintLoss(tf.keras.losses.Loss):
  def call(self, y_true, y_pred):
    # custom loss L_orient = 0.5*(q [-] \hat{q})^T *Sig^{-1}* 
    q = tf.slice(y_true,
               begin=[0,0],
               size=[4,64])                     ##y_true[0:4][:]
    #print(tf.shape(q))

    q_est = tf.slice(y_pred,
               begin=[0,0],
               size=[4,64])   #########y_pred[0:4][:]

    delta = boximinus(q, q_est)
    print(tf.shape(delta))

    sig = tnp.eye(3)

    return tf.convert_to_tensor(tnp.matmul(tnp.matmul(tnp.transpose(delta), sig), delta))

class MyLossP(tf.keras.losses.Loss):
  def call(self, y_true, y_pred):
    # y_pred = tf.convert_to_tensor_v2(y_pred)
    # y_true = tf.cast(y_true, y_pred.dtype)
    return tf.sqrt(tf.reduce_mean((y_pred - y_true)**2, axis=-1))





In [17]:
class OreintNet():
    def build_model(self):
        # Building Network
        #1. Define inputs
        Acc_input = Input(shape=(100,3), batch_size=64 , name = 'Acc_input')    
        Gyro_input = Input(shape=(100,3),batch_size=64   , name = 'Gyro_input')
        Mag_input = Input(shape=(100,3), batch_size=64   , name = 'Mag_input')
        
        MergedLayer = Concatenate()([Acc_input , Gyro_input, Mag_input])
        #
        ##MergedLayer = Input(shape=(9) , name = 'imu_input')
        #2. LSTM
        LSTM1 = (LSTM(100, return_sequences=True))(MergedLayer) # , return_sequences=True
        LSTM2 = (LSTM(100, return_sequences=False))(LSTM1)
        #print("Output shape of the LSTM layer's output : ", LSTM2.shape)
        
        #3. Fully-Connected (Sigma)
        Dense1 = Dense(units=100, input_shape=(64,100), activation='tanh')(LSTM2)
        Dense2 = Dense(units=32, activation='tanh')(Dense1)
        theta_output = Dense(units=4, name='theta_out')(Dense2) # Sigma Outputs
        
        #4. Fully-Connected (Theta)
        Dense3 = Dense(units=100, input_shape=(64,100), activation='tanh')(LSTM2) # input_shape=(
        Dense4 = Dense(units=32, activation='tanh')(Dense3)
        sigma_output = Dense(units=6, name='sig_out')(Dense4) # Theta Outputs  activation='linear'


        output = Concatenate()([theta_output, sigma_output])
        #5. Define and compile The model
        Network = Model([Acc_input,Gyro_input, Mag_input],  [output])  #  [Acc_input,Gyro_input, Mag_input]
        Network.compile(loss=MyLossP(), optimizer=tf.keras.optimizers.Adam(learning_rate=orient_learning_rate) ,metrics=['mape']) #loss=OreintLoss()
        return Network
        

In [18]:
#class OreintNet(Model):
    

In [19]:
network_builder = OreintNet()
Nepochs = 20
if initial_epoch > 0:
  print('continue after epoch' , initial_epoch , ' - checkpoint: ',latest_check_point,' epochs = ',Nepochs)
  NavNet = tf.keras.models.load_model(os.path.join(checkpoints_path,latest_check_point),compile=False)
  NavNet.compile(loss=MyLossP(), optimizer=tf.keras.optimizers.Adam(learning_rate=orient_learning_rate) ,metrics=['mape'])
else:
  print('start from scratch : epochs = ',Nepochs)
  orientation_network = network_builder.build_model()

start from scratch : epochs =  20


2021-11-25 20:40:05.607664: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-25 20:40:05.608237: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-25 20:40:05.608447: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-25 20:40:05.608590: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [20]:
orientation_network.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Acc_input (InputLayer)          [(64, 100, 3)]       0                                            
__________________________________________________________________________________________________
Gyro_input (InputLayer)         [(64, 100, 3)]       0                                            
__________________________________________________________________________________________________
Mag_input (InputLayer)          [(64, 100, 3)]       0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (64, 100, 9)         0           Acc_input[0][0]                  
                                                                 Gyro_input[0][0]             

In [21]:

history = orientation_network.fit(
    generate_training_samples(acc_np, gyro_np, mag_np, orient_np),
    epochs=Nepochs,
    initial_epoch=initial_epoch,
    steps_per_epoch=len(orient_np),
    batch_size=64,
    callbacks = [check_point_callback]
)

Epoch 1/20


2021-11-25 20:40:06.457507: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-25 20:40:08.213747: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201


  1851/849119 [..............................] - ETA: 2:02:33 - loss: 0.0057 - mape: 4244850.5000

In [ ]:
pos_learning_rate = 0.001